In [28]:
import nest_asyncio
nest_asyncio.apply()

In [29]:
import tensorflow as tf
import tensorflow_federated as tff

In [30]:
import os
import random
import numpy as np

from collections import deque

from keras.layers import Input, Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam

from DRL import DRL
from DQN import DQN

In [31]:
NUM_CLIENTS = 3
BATCH_SIZE = 100


In [32]:
import gym
import numpy as np
from easydict import EasyDict

import ray
from ray.tune.registry import register_env
from ray import tune
from ray.rllib.env.multi_agent_env import MultiAgentEnv

def make_multiagent(args):
    class MultiEnv(MultiAgentEnv):
        def __init__(self):
            self.agents = [gym.make(args.env) for _ in range(args.num_agents)]
            self.dones = set()
            self.observation_space = self.agents[0].observation_space
            self.action_space = self.agents[0].action_space

        def reset(self):
            self.dones = set()
            return {i: a.reset() for i, a in enumerate(self.agents)}

        def step(self, action_dict):
            obs, rew, done, info = {}, {}, {}, {}
            for i, action in action_dict.items():
                obs[i], rew[i], done[i], info[i] = self.agents[i].step(action)
                if done[i]:
                    self.dones.add(i)
            done["__all__"] = len(self.dones) == len(self.agents)
            return obs, rew, done, info

    return MultiEnv

def make_fed_env(args):   
    FedEnv = make_multiagent(args)
    env_name = "multienv_FedRL"
    register_env(env_name, lambda _: FedEnv())
    return env_name

def gen_policy_graphs(args):
    single_env = gym.make(args.env)
    obs_space = single_env.observation_space
    act_space = single_env.action_space
    policy_graphs = {f'agent_{i}': (None, obs_space, act_space, {}) 
         for i in range(args.num_agents)}
    return policy_graphs

def policy_mapping_fn(agent_id):
    return f'agent_{agent_id}'
def change_weights(weights, i):
    """
    Helper function for FedQ-Learning
    """
    dct = {}
    for key, val in weights.items():
        # new_key = key
        still_here = key[:6]
        there_after = key[7:]
        # new_key[6] = i
        new_key = still_here + str(i) + there_after
        dct[new_key] = val
    # print(dct.keys())
    return dct

def synchronize(agent, weights, num_agents):
    """
    Helper function to synchronize weights of the multiagent
    """
    weights_to_set = {f'agent_{i}': weights 
         for i in range(num_agents)}
    # weights_to_set = {f'agent_{i}': change_weights(weights, i) 
    #    for i in range(num_agents)}
    # print(weights_to_set)
    agent.set_weights(weights_to_set)

def uniform_initialize(agent, num_agents):
    """
    Helper function for uniform initialization
    """
    new_weights = agent.get_weights(["agent_0"]).get("agent_0")
    # print(new_weights.keys())
    synchronize(agent, new_weights, num_agents)

def compute_softmax_weighted_avg(weights, alphas, num_agents, temperature=1):
    """
    Helper function to compute weighted avg of weights weighted by alphas
    Weights and alphas must have same keys. Uses softmax.
    params:
        weights - dictionary
        alphas - dictionary
    returns:
        new_weights - array
    """
    def softmax(x, beta=temperature, length=num_agents):
        """Compute softmax values for each sets of scores in x."""
        e_x = np.exp(beta * (x - np.max(x)))
        return (e_x / e_x.sum()).reshape(length, 1)
    
    alpha_vals = np.array(list(alphas.values()))
    soft = softmax(alpha_vals)
    weight_vals = np.array(list(weights.values()))
    new_weights = sum(np.multiply(weight_vals, soft))
    return new_weights

def reward_weighted_update(agent, result, num_agents):
    """
    Helper function to synchronize weights of multiagent via
    reward-weighted avg of weights
    """
    return softmax_reward_weighted_update(agent, result, num_agents, temperature=0)

def softmax_reward_weighted_update(agent, result, num_agents, temperature=1):
    """
    Helper function to synchronize weights of multiagent via
    softmax reward-weighted avg of weights with specific temperature
    """
    all_weights = agent.get_weights()
    policy_reward_mean = result['policy_reward_mean']
    episode_reward_mean = result['episode_reward_mean']
    if policy_reward_mean:
        new_weights = compute_softmax_weighted_avg(all_weights, policy_reward_mean, num_agents, temperature=temperature)
        synchronize(agent, new_weights, num_agents)

def fed_train(args):
    temp_schedule = args.temp_schedule
    temperature = temp_schedule[0]
    hotter_temp = temp_schedule[1]
    temp_shift = temp_schedule[2]
    fed_schedule = args.fed_schedule
    num_iters = fed_schedule[0]
    increased_iters = fed_schedule[1]
    fed_shift = fed_schedule[2]
    
    num_agents = args.num_agents
    def fed_learn(info):
#       get stuff out of info
        result = info["result"]
        agent = info["trainer"]
        optimizer = agent.optimizer
        if result['timesteps_total'] > fed_shift:
            num_iters = increased_iters
        if result['timesteps_total'] > temp_shift:
            temperature = hotter_temp
        # correct result reporting
        result['episode_reward_mean'] = result['episode_reward_mean']/num_agents
        result['episode_reward_max'] = result['episode_reward_max']/num_agents
        result['episode_reward_min'] = result['episode_reward_min']/num_agents
        result['federated'] = "No federation"
        if result['training_iteration'] == 1:
            uniform_initialize(agent, num_agents)
        elif result['training_iteration'] % num_iters == 0:
            result['federated'] = f"Federation with {temperature}"
            # update weights
            softmax_reward_weighted_update(agent, result, num_agents, temperature)
            # clear buffer, don't want smoothing here
            optimizer.episode_history = []
    return fed_learn

def fedrl(args):
    ray.init(ignore_reinit_error=True)
    policy_graphs = gen_policy_graphs(args)
    multienv_name = make_fed_env(args)
    callback = fed_train(args)
    tune.run(
        args.algo,
        name=f"{args.env}-{args.algo}-{args.num_agents}",
        stop={"episode_reward_mean": 500},
        config={
                "multiagent": {
                    "policy_graphs": policy_graphs,
                    "policy_mapping_fn": tune.function(lambda agent_id: f'agent_{agent_id}'),
                },
                "env": multienv_name,
                "gamma": 0.99,
                # "lambda": 0.95,
                # "kl_coeff": 1.0,
                # "num_sgd_iter": 32,
                # "lr": .0003 * args.num_agents,
                # "vf_loss_coeff": 0.5,
                # "clip_param": 0.2,
                # "sgd_minibatch_size": 4096,
                "train_batch_size": 65536,
                # "grad_clip": 0.5,
                # "batch_mode": "truncate_episodes",
                # "observation_filter": "MeanStdFilter",
                # "lr": tune.grid_search(args.lrs),
#                 "simple_optimizer": True,
                "callbacks":{
                    "on_train_result": tune.function(callback),
                },
                "num_workers": args.num_workers,
                "num_gpus": 0,
            },
        checkpoint_at_end=False
    )


args = EasyDict({
    'num_agents': 5,
    'num_workers': 7,
    'fed_schedule': [1, 5, 2e7],
    # 'temperatures': [0, 8, 0.5, 4, 2, 1, 16],
    'temp_schedule': [0.5, 2, 2e7],
    # 'timesteps': 1e7,
    # 'lr': 5e-4,
    # 'lrs': [5e-5, 5e-4, 5e-3],
    # 'episodes': 150,
#     'num_iters': 100,
    'env': 'CartPole-v0',
    'name': 'fed_experiment',
    'algo': 'PG',
})
# train
fedrl(args)
# eval

2021-12-08 18:57:17,724	INFO worker.py:833 -- Calling ray.init() again after it has already been called.
2021-12-08 18:57:17,732	WARNING sample.py:403 -- DeprecationWarning: wrapping <function fedrl.<locals>.<lambda> at 0x7fc81e56a598> with tune.function() is no longer needed
2021-12-08 18:57:17,733	WARNING sample.py:403 -- DeprecationWarning: wrapping <function fed_train.<locals>.fed_learn at 0x7fc80dd10c80> with tune.function() is no longer needed


Trial name,status,loc
PG_multienv_FedRL_9c0e2_00000,PENDING,


(raylet) /Users/a10.11.5/anaconda3/lib/python3.7/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(raylet)   warnings.warn(msg)
(pid=33654) 2021-12-08 18:57:33,537	WARNING compression.py:16 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(pid=33654) 2021-12-08 18:57:35,282	INFO trainer.py:753 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=33654) 2021-12-08 18:57:35,283	WARNING deprecation.py:39 -- DeprecationWarning: `callbacks dict interface` has been deprecated. Use `a class extending rllib.agents.callbacks.DefaultCallbacks` instead. This will raise an error in the future!
(pid=33654) 2021-12-08 18:57:35,283	INFO trainer.py:772 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(raylet) /Users/a10.11.5/anaconda3/lib/python3.7/s

Trial name,status,loc
PG_multienv_FedRL_9c0e2_00000,RUNNING,127.0.0.1:33654


2021-12-08 18:58:00,142	ERROR trial_runner.py:924 -- Trial PG_multienv_FedRL_9c0e2_00000: Error processing event.
Traceback (most recent call last):
  File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 890, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 788, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/worker.py", line 1625, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(KeyError): ray::PG.train_buffered() (pid=33654, ip=127.0.0.1, repr=PG)
  File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/tune/trainable.py", line 224, in train_buf

...

(pid=33654) 2021-12-08 18:58:00,133	ERROR worker.py:80 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::RolloutWorker.par_iter_next() (pid=33671, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7faf311621d0>)
(pid=33654)   File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/util/iter.py", line 1151, in par_iter_next
(pid=33654)     return next(self.local_it)
(pid=33654)   File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 378, in gen_rollouts
(pid=33654)     yield self.sample()
(pid=33654)   File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 753, in sample
(pid=33654)     batches = [self.input_reader.next()]
(pid=33654)   File "/Users/a10.11.5/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/sampler.py", line 103, in next
(pid=33654)     batches = [self.get_data()]
(pid=33654)   File "/Users/a10.11.5/

Trial name,status,loc
PG_multienv_FedRL_9c0e2_00000,ERROR,127.0.0.1:33654
Trial name,# failures,error file
PG_multienv_FedRL_9c0e2_00000,1,/Users/a10.11.5/ray_results/CartPole-v0-PG-5/PG_multienv_FedRL_9c0e2_00000_0_2021-12-08_18-57-17/error.txt


Trial name,status,loc
PG_multienv_FedRL_9c0e2_00000,ERROR,127.0.0.1:33654
Trial name,# failures,error file
PG_multienv_FedRL_9c0e2_00000,1,/Users/a10.11.5/ray_results/CartPole-v0-PG-5/PG_multienv_FedRL_9c0e2_00000_0_2021-12-08_18-57-17/error.txt


(pid=33658) [2021-12-08 18:58:00,784 E 33658 2054927] raylet_client.cc:159: IOError: Broken pipe [RayletClient] Failed to disconnect from raylet.
(pid=33671) [2021-12-08 18:58:00,786 E 33671 2055014] raylet_client.cc:159: IOError: Broken pipe [RayletClient] Failed to disconnect from raylet.
(pid=33659) [2021-12-08 18:58:00,784 E 33659 2054930] raylet_client.cc:159: IOError: Broken pipe [RayletClient] Failed to disconnect from raylet.
(pid=33660) [2021-12-08 18:58:00,784 E 33660 2054945] raylet_client.cc:159: IOError: Broken pipe [RayletClient] Failed to disconnect from raylet.


TuneError: ('Trials did not complete', [PG_multienv_FedRL_9c0e2_00000])

In [ ]:
models_ = [[]]
num_models = 3
beta = 0.5
params = models_[0].named_parameters()
dict_params = dict(params)

import torch 


params = models_[0].named_parameters()
def model_aggregate(self, weight_accumulator):
        for i in range(1, num_models):
            for name, param in models_[i].named_parameters():
                if name in dict_params:
                    mod_eval = [dict_params[name]/dict_params.sum()]
                    dict_params[name].data.copy_(param.data*mod_eval[name])
        